In [1]:
import pandas as pd
import os
from env import get_db_url
import matplotlib as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.preprocessing


### Mall Customers

- Acquire data from the customers table in the mall_customers database.
- Summarize the data (include distributions and descriptive statistics).
- Detect outliers using IQR.
- Split data into train, validate, and test.
- Encode categorical columns using a one hot encoder (pd.get_dummies).
- Handles missing values.
- Scaling

### Acquire data from mall_customers database

In [ ]:
sql = '''SELECT * FROM customers;'''
mall = pd.read_sql(sql, get_db_url('mall_customers'));

In [ ]:
mall.set_index(['customer_id'], inplace=True)

### Summarize the data

In [ ]:
mall.head()

In [ ]:
mall.info()

In [ ]:
mall.describe().T

In [ ]:
mall.hist(figsize=(18,10),bins=10)

### Detect outliers

In [ ]:
sns.boxplot(y=mall.age)

In [ ]:
# annual income shows outliers
sns.boxplot(y=mall.annual_income)

In [ ]:
sns.boxplot(y=mall.spending_score)

In [ ]:
q1 = mall.annual_income.quantile(0.25)
q3 = mall.annual_income.quantile(0.75)
iqr = q3 - q1
upper_bound =  q3 + 1.5 * iqr


In [ ]:
# there are 2 outliers at 137k for annual income
mall[mall.annual_income > upper_bound]

### Split the data

In [ ]:
def split_mall_data(df):
    ''' This function splits the cleaned dataframe into train, validate, and test 
    datasets.'''

    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123) 
                                   
    return train, validate, test

In [ ]:
train, validate, test = split_mall_data(mall)

### Encode the data

In [ ]:
dummy_df = pd.get_dummies(mall[['gender']],dummy_na=False, drop_first=True)

In [ ]:
mall = pd.concat([mall, dummy_df], axis=1)

In [ ]:
mall = mall.drop(columns='gender')

In [ ]:
mall.head()

### Handle missing values

In [ ]:
# no missing values to handle
mall.isna().sum()

### Scale the data

In [ ]:
def scale_data(train,
              validate,
              test,
              columns_to_scale=['age','annual_income', 'spending_score']):
    '''
    Scales the split data.
    Takes in train, validate and test data and returns the scaled data.
    '''
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    
    #using MinMaxScaler (best showing distribution once scaled)
    scaler = sklearn.preprocessing.MinMaxScaler()
    scaler.fit(train[columns_to_scale])
    
    #creating a df that puts MinMaxScaler to work on the wanted columns and returns the split datasets and counterparts
    train_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(train[columns_to_scale]),
                                                 columns=train[columns_to_scale].columns.values).set_index([train.index.values])
    
    validate_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(validate[columns_to_scale]),
                                                 columns=validate[columns_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[columns_to_scale] = pd.DataFrame(scaler.transform(test[columns_to_scale]),
                                                 columns=test[columns_to_scale].columns.values).set_index([test.index.values])
    
    
    return train_scaled, validate_scaled, test_scaled

In [ ]:
train_scaled, validate_scaled, test_scaled = scale_data(train,test,validate)

In [ ]:
train_scaled.head()

# To Do: wrangle_mall.py